In [12]:
import copy
import time
import nltk
import random
import gensim
import pickle
import pandas as pd
import numpy as np
import lda
import lda.datasets
import collections
import brisk
from numba import jit
from scipy.spatial.distance import pdist
np.set_printoptions(suppress=True)

In [13]:
n_topics = 120  # topic number
rng = np.random.mtrand._rand
rands = rng.rand(1024 ** 2 // 8) # rand seeds

# Load the model
path_dir = './dataset/'
datasets = 'nips'

with open(path_dir + datasets + '/gibbs_lda_model.pickle','rb') as file:
    model = pickle.loads(file.read())

# Load the vocab
path = path_dir + datasets + '/data/'
vocab = lda.datasets.load_reuters_vocab(path)
voc = collections.defaultdict()
for index, i in enumerate(vocab):
    word = i.replace('\n', '')
    if not word:
        continue
    voc[word] = index

# Load test set and candidate set
we = pd.read_csv(path_dir + datasets +  '/pre-processing.csv')
we['similar_words'] = we['similar_words'].map(lambda x: eval(x))

def eval_string(x):
    x = x.replace('[','').replace(']','').replace(',','')
    x = x.strip().split()
    x = [float(i) for i in x]
    return x
we['word_embedding'] = we['word_embedding'].map(lambda x:eval_string(x))

def evals(x):
    if x == '<class \'list\'>':
        return []
    else:
        return eval(x)
we['similar_mixed'] = we['similar_mixed'].map(lambda x: evals(x))

f_s = open(path_dir + datasets + '/submissions.csv')
submissions = pd.read_csv(f_s)
submissions['paper_text_tokens'] = submissions['paper_text_tokens'].map(lambda x: eval(x))

# Load word embedding (it will cost a lot of time)
word_embedding_model = gensim.models.KeyedVectors.load_word2vec_format(path_dir + 'wikien.vec')

# Load Bert
from bert_serving.client import BertClient
bc = BertClient()

KeyboardInterrupt: 

In [ ]:
@jit(nopython=True,parallel=True)
def infer(nzw, x_index, n_iter=500, m_alpha=0.1, m_eta=0.01):
    '''
    infer the new documents 
    ----------
    Input
        nzw: the results of LDA model after training stage
        x_index: array-like, shape(, n_features)
            Testing vector, where n_features is the number of features.
        n_iter: iterations, default 500
        m_alpha: alpha, gibbs LDA parameter, default 0.1
        m_eta: eta, gibbs LDA parameter, default 0.01
    Output
        Topic distribution of the test document 
    '''
    global n_topics
    n_rand = 131072  # a large number
    alpha = np.repeat(m_alpha, n_topics).astype(np.float64)
    eta = np.repeat(m_eta, len(voc)).astype(np.float64)
    eta_sum = np.sum(eta) 

    # Initialize
    nz = np.zeros(n_topics)
    ZS = np.empty(len(x_index), np.int32)
    dist_sum = np.zeros(n_topics)
    for index, w in enumerate(x_index):
        z_new = index % n_topics
        ZS[index] = z_new
        nzw[z_new, w] += 1
        nz[z_new] += 1
    # Sampling
    for it in range(n_iter):
        for index, w in enumerate(x_index):
            z = ZS[index]
            nzw[z, w] -= 1
            nz[z] -= 1
            dist_cum = 0
            for k in range(n_topics):
                dist_cum += (nzw[k, w] + eta[w]) / (nz[k] + eta_sum) * (nz[k] + alpha[k])
                dist_sum[k] = dist_cum

            r = rands[index % n_rand] * dist_cum 
            z_new = brisk.bisect_left(dist_sum, r)
            ZS[index] = z_new
            nzw[z_new, w] += 1
            nz[z_new] += 1

    return nz

In [ ]:
def calculate_diff(word_count, voc, Nkv, target):
    diff = 0
    for (word, count) in word_count.items():
        wi = voc[word]
        diff += count * (Nkv[:, wi][target]) / brisk.sum(Nkv[:, wi])

    return diff

def rank_cal(x, target, result, sim_w, sim_s, flag):
    new_x = x / sum(x)
    output = {}
    for index, k in enumerate(new_x):
        output[index] = k

    output = sorted(output.items(), key = lambda x:x[1], reverse = True)
    for index, k in enumerate(output):
        if k[0] != target and k[1] == 0:
            result[flag].append([index + 1, k[0], k[1], sim_w, sim_s])
            break
        if k[0] == target:
            result[flag].append([index + 1, k[0], k[1], sim_w, sim_s])
            break
    return result

In [ ]:
# Part of speech screening, see http://www.nltk.org/_modules/nltk/tag.html for more details.
wordq = [['JJ', 'JJR', 'JJS'], ['NN', 'NNS', 'NNP', 'NNPS'], 
         ['PRP', 'PRP$'], ['RB', 'RBR', 'RBS'],
         ['VB', 'VBD', 'VBG', 'VBN', 'VBZ']]
        
def pos_filter(x, word):
    if x[0][0] == word:
        del x[0]
    pos_list = nltk.pos_tag(np.array(x)[:, 0].tolist())
    pos_list = np.array(pos_list)[:, 1]
    
    target = nltk.pos_tag([word])
    target_list = []
    for i in wordq:
        if target[0][1] in i:
            target_list = i

    index = [k for k in range(len(np.array(pos_list))) if (pos_list[k] in target_list)]
    res = [tuple(p) for p in np.array(x)[index]]
    return res

In [ ]:
def get_similar_words(word_index, word, Nkv, new):
    em_word_index = int(list(new[new['word'].isin([word])].index.values)[0])
    ori_phi = Nkv[:, word_index] / sum(Nkv[:, word_index])
    similar_words = new.loc[em_word_index, 'similar_mixed']
    similar_words = pos_filter(similar_words, word)
    return similar_words, ori_phi

In [ ]:
# Combine the 5 words before and after the current word to form a sentence
def merge_sentence(pos, doc, word_candidate):
    if pos >= 5 and pos <= len(doc) - 5:
        ori_sentence = ' '.join(doc[pos - 5: pos + 5])  # combine sentence
        adv_sentence = ' '.join(doc[pos - 5: pos]) + ' ' + word_candidate[1] + ' ' + ' '.join(doc[pos + 1: pos + 5])
    elif pos < 5:
        ori_sentence = ' '.join(doc[:10])
        adv_sentence = ' '.join(doc[0: pos]) + ' ' + word_candidate[1] + ' ' + ' '.join(doc[pos + 1: 10])
    else:
        ori_sentence = ' '.join(doc[-10:])
        adv_sentence = ' '.join(doc[-10: pos]) + ' ' + word_candidate[1] + ' ' + ' '.join(doc[pos + 1:])
    
    return ori_sentence, adv_sentence

In [ ]:
def seak_for_word(doc, target, nd, Nkv, length, Att_algo, t, alpha = 0.005):
    '''
    Input
        doc: the victim submission
        target: the target topic
        nd: the predicted topic sampling count of the victim submission
        Nkv: the training topic-word sampling count of whole voc
        length: the length of original victim submission
        Att_algo: attack algorithm, different level parameter
        t: the unique words substitution parameter
        alpha: modification budget, default 0.005
    Output
        attack path, adversarial topic sampling count estimation, adversarial submission
    '''
    global voc
    global path
    solution_space = []  # Get solution space
    for i in sub_sample:
        if i in we['word'].values and i not in solution_space:
            solution_space.append(i)

    new_we = we[we['word'].isin(solution_space)]
    new = copy.deepcopy(new_we)
    iterations = int(length * alpha)
    nd = nd.astype('float64')
    word_dis, sentence_dis = 0.0, 0.0  # word distortion, sentence distortion
    adv_solu = {}  # attack path
    already = []
    vic_visited = []
    word_counts = collections.Counter(doc)  # word counts
    d_v = 0.4  # word embedding distance threshold
    cur_change = 0  # current modification count

    word_contribution = collections.defaultdict(list)  # word-phi dict
    contributions = 0.0
    for index, word in enumerate(list(set(doc))):
        word_index = voc[word]
        phi = Nkv[:, word_index] / sum(Nkv[:, word_index])
        word_contribution[word_index] = phi[target]
        contributions += (word_contribution[word_index] * word_counts[word])

    if 'baseline' in Att_algo:
        if Att_algo != 'baseline_4':
            victims = random.sample(range(len(doc)), iterations)
        if Att_algo == 'baseline_3' or Att_algo == 'baseline_4':
            attack_words = (-model.topic_word_[target]).argsort()[:]
            base3iter = 0
            base4iter = 0
            att_words = []
        for i in range(iterations):
            if Att_algo == 'baseline_4':
                base4flag = 0
                while base4flag == 0:
                    target_index = attack_words[base4iter]
                    sub_word = list(voc.keys())[list(voc.values()).index(target_index)]
                    if sub_word in att_words:
                        base4iter += 1
                        continue
                    # get the most similar words in doc
                    sw_list = word_embedding_model.most_similar_to_given_top(sub_word, list(set(doc)), topk=len(list(set(doc))))
                    # pos filter
                    sw_list = pos_filter(sw_list, sub_word)
                    for k_word, k_sim in sw_list:
                        victim, sim_value = k_word, float(k_sim)
                        if sim_value < 1 - d_v:
                            continue
                        sim_value = 1 - sim_value
                        victim_index = [ind for ind, x in enumerate(doc) if ((x == victim) and (ind not in adv_solu.keys()))][0]
                        if victim_index not in already:
                            base4flag = 1
                            att_words.append(sub_word)
                            break
                    if base4flag == 0:
                        base4iter += 1    
            else:
                victim_index = victims[i]
                victim = doc[victim_index]
                victim_pos = nltk.pos_tag([victim])[0][1]
                for p in wordq:
                    if victim_pos in p:
                        break

                if Att_algo == 'baseline_1':
                    sub_word = random.choice(np.array(new_we['word'].tolist()))
                    while (nltk.pos_tag([sub_word])[0][1] not in p) or (sub_word == victim):
                        sub_word = random.choice(np.array(new_we['word'].tolist()))
                    
                    sim_value = (1 - pdist([np.array(new_we[new_we['word'].isin([sub_word])]['word_embedding'].tolist()[0]), 
                                            np.array(new_we[new_we['word'].isin([victim])]['word_embedding'].tolist()[0])],
                                            'cosine'))[0]
                    sim_value = 1 - sim_value

                elif Att_algo == 'baseline_2':
                    target_list = new_we[new_we['word'].isin([victim])]['similar_words'].tolist()[0]
                    for tar in target_list:
                        if nltk.pos_tag([tar[0]])[0][1] not in p:
                            continue
                    sub_word, sim_value = tar
                    sim_value = float(sim_value)

                elif Att_algo == 'baseline_3':
                    target_index = attack_words[base3iter]
                    sub_word = list(voc.keys())[list(voc.values()).index(target_index)]

                    while (nltk.pos_tag([sub_word])[0][1] not in p) or (sub_word == victim) or (sub_word in att_words):
                        base3iter += 1
                        target_index = attack_words[base3iter]
                        sub_word = list(voc.keys())[list(voc.values()).index(target_index)]

                    sim_value = word_embedding_model.similarity(sub_word, victim)
                    sim_value = 1 - sim_value
                    att_words.append(sub_word)

            word_candidate = (victim, sub_word, 0, victim_index, float(sim_value))     
            pos = victim_index
            
            ori_sentence, adv_sentence = merge_sentence(pos, doc, word_candidate)
             # Calculate ASSD
            ori_sentence_em = bc.encode([ori_sentence])[0]
            adv_sentence_em = bc.encode([adv_sentence])[0]
            sentence_dis += pdist([ori_sentence_em, adv_sentence_em], 'cosine')
            
            doc[victim_index] = word_candidate[1]  # modificate 
            cur_change += 1
            adv_solu[(victim_index, word_candidate[0])] = word_candidate[1]  # save attack path
            already.append(word_candidate[3])
            word_dis += word_candidate[-1]
    else:  # EvaLDA
        while (cur_change < iterations):
            word_cos_max = float('-inf')
            for index, word in enumerate(list(set(doc))):  # find victim word
                if word in vic_visited:
                    continue
                word_index = voc[word]  # victim word's index
                if word not in new['word'].values or word_counts[word] < 1:
                    continue
                similar_words, ori_phi = get_similar_words(word_index, word, Nkv, new)
                if similar_words == []:  # wordnet may don't have similar words
                    continue          
                for (sim_word, sim_value) in similar_words:
                    if (sim_word not in voc) or float(sim_value) < 1 - d_v:  # distance budget
                        continue
                    sim_index = voc[sim_word]
                    new_phi = Nkv[:, sim_index] / sum(Nkv[:, sim_index])
                    if t >= (iterations - cur_change):  # calculate the real modification budget
                        tmp = iterations - cur_change
                    else:
                        tmp = t
                    real_t = min(tmp, word_counts[word])
                    tem = contributions + (new_phi[target] - word_contribution[word_index]) * real_t
                    score = tem ** int(Att_algo[-1]) - contributions ** int(Att_algo[-1])
                    if score > word_cos_max: # update word_candidate
                        word_cos_max = score
                        sim_value = (1 - float(sim_value)) 
                        word_candidate = (word, sim_word, score, index, float(sim_value), real_t)
                    elif score == word_cos_max:
                        if word_candidate and word_candidate[-2] > (1 - float(sim_value)):
                            sim_value = (1 - float(sim_value)) 
                            word_cos_max = score
                            word_candidate = (word, sim_word, score, index, float(sim_value), real_t)   
            
            pos_candidate = []
            for ind, x in enumerate(doc):
                if (x == word_candidate[0]) and (ind not in already):
                    pos_candidate.append(ind)
            p_count = 0  # current modification of the unique word
            for pos in pos_candidate:
                ori_sentence, adv_sentence = merge_sentence(pos, doc, word_candidate)
                ori_sentence_em = bc.encode([ori_sentence])[0]
                adv_sentence_em = bc.encode([adv_sentence])[0]
                sentence_dis += pdist([ori_sentence_em, adv_sentence_em], 'cosine')  # ASSD

                doc[pos] = word_candidate[1]  # modificate the victim document
                adv_solu[(pos, word_candidate[0])] = word_candidate[1]  # save attack path
                already.append(pos)
                word_dis += word_candidate[-2]
                word_counts[word_candidate[0]] -= real_t
                word_counts[word_candidate[1]] += real_t
                # update contributions
                contributions += (Nkv[:, voc[word_candidate[1]]] / sum(Nkv[:, voc[word_candidate[1]]]))[target] \
                                 - word_contribution[voc[word_candidate[0]]]
                cur_change += real_t
                p_count += 1
                if p_count == word_candidate[-1]:
                    break;
                if cur_change == iterations: 
                    break
            vic_visited.append(word_candidate[0])

    word_dis /= cur_change  # take average
    sentence_dis /= cur_change 
    return adv_solu, nd, doc, word_dis, sentence_dis

In [ ]:
alpha = 0.01  # modification threshold
k = 1 # modify the unique word in k positions in each iteration
test_list = list(range(0, submissions.index.size))
random.shuffle(test_list) # you should record this test_list to ensure the same test samples
test_iterations = 50 # run 50 samples
result = collections.defaultdict(list)
target_list = {0: 19, 1: 14, 2: 9, 3: 4} # original topic rankings, counting from 0.
total_result = []
total_path = []
for t in range(4):  # different rank_cal
    result = collections.defaultdict(list)
    Path = collections.defaultdict(list)
    for i in range(test_iterations):
        sub_sample = submissions.iloc[test_list[i]]['paper_text_tokens']

        length = len(sub_sample)
        del_list = []
        for index, w in enumerate(sub_sample):
            if w not in voc or w not in word_embedding_model:
                del_list.append(index)
            else:
                continue

        sub_sample = [sub_sample[i] for i in range(0, len(sub_sample),
            1) if i not in del_list]
        word_index_list = []
        for word in sub_sample:
            word_index_list.append(voc[word]) 
        if word_index_list == []:
            continue
        '''
        -----------------Before attack-----------------
        '''
        n_d_sub = infer(model.nzw_, word_index_list)
        d_t_sub = n_d_sub / sum(n_d_sub)

        output_new = {}
        for index, i in enumerate(d_t_sub):
            output_new[index] = i

        output_new = sorted(output_new.items(), key=lambda x:x[1], reverse=True)
        common = []
        for j in output_new:
            if j[1] != 0:
                common.append(list(j))
        target = int(np.array(common)[:, 0][target_list[t]])  # target rank
        str_tmp = 'Common_' + str(t + 1)
        for index, i in enumerate(output_new):
            if i[0] == target:
                result[str_tmp].append([index+1, i[0], i[1]])
        '''
        -----------------EvaLDA: level 4-----------------
        '''
        str_tmp = 'level_4_' + str(t + 1)
        adv_path_2, new_nd, new_doc, sim_w, sim_s = \
            seak_for_word(copy.deepcopy(sub_sample), target,
                        n_d_sub, model.nzw_, length, 'level_4',
                        k, alpha=alpha)
        word_index_list = []
        for word in new_doc:
            word_index_list.append(voc[word])
        n_d_new_2 = infer(model.nzw_, word_index_list)
        result = rank_cal(n_d_new_2, target, result, sim_w, sim_s, str_tmp)

        Path[str_tmp].append([adv_path_2])
        '''
        -----------------Baseline 1-----------------
        '''
        str_tmp = 'baseline_1_' + str(t + 1)
        adv_path, new_nd, new_doc, sim_w, sim_s = \
            seak_for_word(copy.deepcopy(sub_sample), target,
                        n_d_sub, model.nzw_, length, 'baseline_1',
                        k, alpha=alpha)
        word_index_list = []
        for word in new_doc:
            word_index_list.append(voc[word])
        n_d_new = infer(model.nzw_, word_index_list)
        result = rank_cal(n_d_new, target, result, sim_w, sim_s, str_tmp)

        Path[str_tmp].append([adv_path])
        '''
        -----------------Baseline 2-----------------
        '''
        str_tmp = 'baseline_2_' + str(t + 1)
        adv_path, new_nd, new_doc, sim_w, sim_s = \
            seak_for_word(copy.deepcopy(sub_sample), target,
                        n_d_sub, model.nzw_, length, 'baseline_2',
                        k, alpha=alpha)
        word_index_list = []
        for word in new_doc:
            word_index_list.append(voc[word])
        n_d_new = infer(model.nzw_, word_index_list)
        result = rank_cal(n_d_new, target, result, sim_w, sim_s, str_tmp)

        Path[str_tmp].append([adv_path])
        '''
        -----------------Baseline 3-----------------
        '''
        str_tmp = 'baseline_3_' + str(t + 1)
        adv_path, new_nd, new_doc, sim_w, sim_s = \
            seak_for_word(copy.deepcopy(sub_sample), target,
                        n_d_sub, model.nzw_, length, 'baseline_3',
                        k, alpha=alpha)
        word_index_list = []
        for word in new_doc:
            word_index_list.append(voc[word])
        n_d_new = infer(model.nzw_, word_index_list)
        result = rank_cal(n_d_new, target, result, sim_w, sim_s, str_tmp)

        Path[str].append([adv_path])
        '''
        -----------------Baseline 4-----------------
        '''
        str_tmp = 'baseline_4_' + str(t + 1)
        adv_path, new_nd, new_doc, sim_w, sim_s = \
            seak_for_word(copy.deepcopy(sub_sample), target,
                        n_d_sub, model.nzw_, length, 'baseline_4',
                        k, alpha=alpha)
        word_index_list = []
        for word in new_doc:
            word_index_list.append(voc[word])
        n_d_new = infer(model.nzw_, word_index_list)
        result = rank_cal(n_d_new, target, result, sim_w, sim_s, str_tmp)

        Path[str_tmp].append([adv_path])
        print(result)
    total_path.append(Path)
    total_result.append(result)

In [ ]:
def res_ans(result):
    res = [[0, 0, 0, 0], [0,0, 0, 0], [0, 0,0, 0], [0,0, 0, 0], [0,0, 0, 0], [0, 0,0, 0]]
    for index, i in enumerate(result):
        for jindex, j in enumerate(result[i]):
            res[index][0] += j[0]
            res[index][1] += j[2]
            res[index][2] += j[-2]
            res[index][3] += j[-1]
    return np.array(res)/len_vas
print(res_ans(result))

In [ ]:
def confidence(thre_result):
    for index, i in enumerate(thre_result):
        print(i)
        a = np.array(thre_result[i][:])[:,0]
        print('95% confidence interval：rank\n', [np.mean(a) - 1.96 * np.std(a) / np.sqrt(len(a)), np.mean(a) + 1.96 * np.std(a) / np.sqrt(len(a))])
        a = np.array(thre_result[i][:])[:,2]
        print('95% confidence interval：scores\n', [np.mean(a) - 1.96 * np.std(a) / np.sqrt(len(a)), np.mean(a) + 1.96 * np.std(a) / np.sqrt(len(a))])
        if index == 0:
            continue
        a = np.array(thre_result[i][:])[:,3]
        print('95% confidence interval：dis_word\n', [np.mean(a) - 1.96 * np.std(a) / np.sqrt(len(a)), np.mean(a) + 1.96 * np.std(a) / np.sqrt(len(a))])
        a = np.array(thre_result[i][:])[:,4]
        print('95% confidence interval：dis_sentence\n', [np.mean(a) - 1.96 * np.std(a) / np.sqrt(len(a)), np.mean(a) + 1.96 * np.std(a) / np.sqrt(len(a))])
confidence(result)